In [1]:
import torch
from torch.utils.data import Dataset, random_split
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os.path
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dev = 'mps' if torch.has_mps else 'cpu'
print(dev)

mps


In [3]:
device = torch.device(dev)
layer_size = 2156
model = nn.Sequential(
    nn.Linear(539, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, 1)
)

In [4]:
model.load_state_dict(torch.load('./jun8_100am.pth'))
model.to(device)
model.eval()

Sequential(
  (0): Linear(in_features=539, out_features=2156, bias=True)
  (1): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=2156, out_features=2156, bias=True)
  (4): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=2156, out_features=2156, bias=True)
  (7): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Linear(in_features=2156, out_features=2156, bias=True)
  (10): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Linear(in_features=2156, out_features=1, bias=True)
)

In [5]:
test_data = pd.read_csv('./csv_artifacts/test_dataish.csv', dtype=np.float32)
trip_ids = pd.read_csv('./csv_artifacts/test_public.csv')
trip_ids = trip_ids[['TRIP_ID']]
all_xy = test_data.to_numpy()
test_input = torch.tensor(all_xy[:,0:539], dtype=torch.float32).to(device)

In [6]:
with torch.no_grad():
    outputs = model(test_input)

In [7]:
outputs = torch.Tensor.cpu(outputs)

In [8]:
output_file = pd.DataFrame(columns=['TRIP_ID', 'TRAVEL_TIME'])
display(output_file)

,TRIP_ID,TRAVEL_TIME


In [9]:
for count, i in enumerate(outputs):
    trip_id = trip_ids.iloc[count].to_string()
    trip_id = trip_id.replace('TRIP_ID', '').strip()
    print(trip_id)
    item = i.item()
    print('Trip ID: {} Result: {}'.format(count+1, item))
    output_file.loc[len(output_file)] = [trip_id, item]

T1
Trip ID: 1 Result: 797.1266479492188
T2
Trip ID: 2 Result: 863.5735473632812
T3
Trip ID: 3 Result: 752.0440063476562
T4
Trip ID: 4 Result: 723.0446166992188
T5
Trip ID: 5 Result: 857.8239135742188
T6
Trip ID: 6 Result: 1204.4349365234375
T7
Trip ID: 7 Result: 845.2596435546875
T8
Trip ID: 8 Result: 738.8314208984375
T9
Trip ID: 9 Result: 1454.7415771484375
T10
Trip ID: 10 Result: 769.6144409179688
T11
Trip ID: 11 Result: 881.4923706054688
T12
Trip ID: 12 Result: 900.9210815429688
T13
Trip ID: 13 Result: 569.3464965820312
T14
Trip ID: 14 Result: 833.7184448242188
T15
Trip ID: 15 Result: 881.4916381835938
T16
Trip ID: 16 Result: 889.5114135742188
T17
Trip ID: 17 Result: 707.7335205078125
T18
Trip ID: 18 Result: 892.1769409179688
T19
Trip ID: 19 Result: 698.8165893554688
T20
Trip ID: 20 Result: 1290.489990234375
T21
Trip ID: 21 Result: 663.9025268554688
T22
Trip ID: 22 Result: 744.5294799804688
T23
Trip ID: 23 Result: 786.220458984375
T24
Trip ID: 24 Result: 973.337646484375
T25
Trip I

In [10]:
output_file.to_csv('submissions_4.csv', index=False)